<a href="https://colab.research.google.com/github/vkrisvasan/FashionAssist/blob/main/RecommendApparel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-aiplatform ragstack-ai --upgrade


In [ ]:
import pandas as pd
import numpy as np
import getpass, os, requests, sys, json, vertexai

In [ ]:
credential_names = ["GCP_PROJECT_ID", "ASTRA_DB_ENDPOINT", "ASTRA_DB_TOKEN","HF_TOKEN"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)

Provide your...GCP_PROJECT_ID··········
Provide your...ASTRA_DB_ENDPOINT··········
Provide your...ASTRA_DB_TOKEN··········
Provide your...HF_TOKEN··········


In [ ]:
from google.colab import userdata, auth, files, drive
from google.cloud import aiplatform

!gcloud config set project {os.environ["GCP_PROJECT_ID"]}
auth.authenticate_user()


Updated property [core/project].


In [ ]:
from datasets import load_dataset
dataset = load_dataset("vkrisvasan/zara_embedding",split='train')


In [ ]:
df = pd.DataFrame.from_dict(dataset)

In [ ]:
df['embeddings1'] = None

In [ ]:
df.shape

(936, 9)

In [ ]:
df.head(2)

,product_name,link,product_images,price,details,category,gender,embeddings,embeddings1
0,WORKOUT T-SHIRT,https://www.zara.com/in/en/workout-t-shirt-p05...,https://static.zara.net/photos///2023/I/0/2/p/...,32.06,T-shirt made of lightweight textured stretch f...,ACTIVEWEAR,men,"[0.00584839704, 0.00366304931, 0.0416106768, 0...",None
1,SPORTS BACKPACK,https://www.zara.com/in/en/sports-backpack-p13...,https://static.zara.net/photos///2023/I/1/2/p/...,64.26,Multi-pocket backpack. Combination of material...,ACTIVEWEAR,men,"[0.00701390672, 0.00680825347, 0.0142998686, 0...",None


In [ ]:
df.dtypes

product_name       object
link               object
product_images     object
price             float64
details            object
category           object
gender             object
embeddings         object
embeddings1        object
dtype: object

In [ ]:
import io
from vertexai.preview.vision_models import MultiModalEmbeddingModel, Image
vertexai.init(project=os.getenv("GCP_PROJECT_ID"))
model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

#Takes an image (URL or image file) and returns it
def get_image(image_source):
    try:
        if image_source.startswith("http://") or image_source.startswith("https://"):
            img_bytes = requests.get(image_source).content
            img = Image(img_bytes)
        else:
            img = Image(image_source)
        return img
    except (requests.RequestException, IOError) as e:
        print(f"Error loading image from {image_source}: {e}")
        return None

#Gets image embeddings using Google Gemini model
def get_img_embeddings(img, text=""):
  embeddings = model.get_embeddings(image=img,
                                    contextual_text=text # passes text along with the image (multimodal)
  )
  return embeddings.image_embedding

In [ ]:

numberofproductwithembedding=0
numberofproductwithoutembedding=0
# Iterate over each row in the DataFrame and update the embeddings1 column with image embedding
#Setting dflen as 30 to quickly test the working of embedding creation. The higher the number throws 400 error. Need to debug reason for error in next iteration
dflen=30
#dflen=len(df)
for i in range(dflen):
  # Check if the embeddings1 column is empty (None) before processing
  if df.loc[i, 'embeddings1'] is None:
    image = get_image(df.loc[i, 'product_images'])
    if image is not None:
      try:
        embeddings1 = get_img_embeddings(image)
        df.at[i, 'embeddings1'] = embeddings1
        numberofproductwithembedding+=1
      except Exception as e:
        #print(f"Error processing image from record #{i}: {df.loc[i, 'product_images']}: {e}")
        numberofproductwithoutembedding+=1
  else:
    numberofproductwithembedding=numberofproductwithembedding+1
print("Product with Embedding",numberofproductwithembedding)
print("Product without Embedding",numberofproductwithoutembedding)

Product with Embedding 30
Product without Embedding 0


In [ ]:

df['embeddings']=df['embeddings'].apply(json.loads)

In [ ]:
from astrapy.db import AstraDB, AstraDBCollection

astra_db = AstraDB(token=os.getenv("ASTRA_DB_TOKEN"),api_endpoint=os.getenv("ASTRA_DB_ENDPOINT"))

In [ ]:
#SET THE correct Dimension based on the embedding model output to avoid error "SHRED_DOC_LIMIT_VIOLATION"
collection = astra_db.create_collection(collection_name="fashion_buddy_workshop_collection",dimension=1408,metric="cosine")
print(f"* Collection: {collection.collection_name}\n")

* Collection: fashion_buddy_workshop_collection



In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
import urllib.request

def load_to_astra(df, collection,dflength):
  len_df=dflength
  errorinfocount=0
  error_info=""
  f=IntProgress(min=0,max=len_df)
  display(f)
  for i in range(len_df):
    f.value += 1
    f.description = str(f.value) + "/" + str(len_df)

    product_name=df.loc[i,"product_name"]
    link = df.loc[i,"link"]
    product_images = df.loc[i,"product_images"]
    price = df.loc[i,"price"]
    details = df.loc[i,"details"]
    category = df.loc[i,"category"]
    gender = df.loc[i,"gender"]
    embeddings = df.loc[i,"embeddings"]

    try:
      collection.insert_one({
          "_id":i,
          "product_name":product_name,
          "link":link,
         "product_images":product_images,
          "price":price,
          "details":details,
          "category":category,
          "gender": gender,
          "$vector": embeddings}) #Keep column name as $Vector to avoid SHRED_DOC_LIMIT_VIOLATION error
    except Exception as error:
      error_info = json.loads(str(error))
      errorinfocount+=1

  print(f"total errors : {errorinfocount} : {error_info}")


In [ ]:
#load_to_astra(df,collection)
dflength = 20# len(df)
load_to_astra(df,collection,dflength)


IntProgress(value=0, max=20)

total errors : 0 : 


In [ ]:
#Takes an image (URL or image file) and returns it
def get_image(image_source):
  if image_source.startswith("http://") or image_source.startswith("https://"):
    img_bytes = requests.get(image_source).content
    img = Image(img_bytes)
    return img
  else:
    print(f" ready to load file {image_source}")
    try:
      with open(image_source, "rb") as f:
        image_bytes = f.read()
      print(f"img_bytes type : {type(image_bytes)}")
      img = Image(image_bytes)
      print(f"img type : {type(img)}")
      return img
    except Exception as error:
      print(f"error loading image from file : {error}" )

#Gets image embeddings using Google Gemini model
def get_img_embeddings(img, text=""):
  embeddings = model.get_embeddings(image=img,
                                    contextual_text=text # passes text along with the image (multimodal)
  )
  print(f"embedding: {embeddings}")
  return embeddings.image_embedding

#Displays recommendations by category returned by Astra Vector Search
def show_recommendations(documents, category=None):
  i=0
  if category:
    print()
    print("Category:", category)
  for doc in documents:
    i+=1
    print(f"\nRecommendation : {i}")
    print(doc["product_name"], "$" + str(doc["price"]), ", Gender:", doc["gender"])
    print(doc["details"])
    print(doc["link"])
    print("Similarity score: ", doc["$similarity"])
    imageurl=doc["product_images"]
    print(imageurl)
    img1=get_image(imageurl)
    imageName1="file1.jpg"
    urllib.request.urlretrieve(imageurl, imageName1)
    #img1 = Image.load_from_file(imageName1)
    display(img1)  # this works but only if it is the only output
    print(img1)
    display(Image(imageurl))
    display(Image("/content/file1.jpg"))
    display(Image(imageName1))
    Image(imageName1)

    Image("/content/file1.jpeg")
    print(imageName1)
    display(Image(imageName1))
    Image(imageurl)

#Finds the similar itens of the given reference image and categories (if any) aleng with a search prompt to give t
def find_similar_items(reference_image, categories=None):
  reference_img = get_image(reference_image)
  print("Reference image:")
  #if reference_img is not None:
    #reference_img.show()
  print("Selected Categories:", ", ".join(categories) if categories else "None")
  if categories:
    for category in categories:
      # Run multiple ANN searches to pull top 3 matches
      # for each category requested
      search_prompt= """
      I am trying to find pieces of apparel that are similar to what is in thIS picture.
      Ignore the-model and only focus on finding the most similar clothing.

      I only care about apparel that falls within the category contained within triple backticks:
      '''{category}'''
      """.format(category=category)
      reference_embeddings = get_img_embeddings(reference_img, text=search_prompt)
      #Run an ANN search filtering by apparel category
      documents = collection.vector_find(reference_embeddings,limit=3,filter={"category": category},include_similarity="True")
      show_recommendations(documents, category=category)
  else:
    search_prompt = """
      I am trying to find pieces of apparel that are similar to what is in this picture.
      Pretend as if there is no model in the image, only clothing.
      """
    print("going to call get_img_embeddings")
    reference_embeddings = get_img_embeddings(reference_img, text=search_prompt)
    #print(reference_embeddings)
    #Run single ANN search across entire DB
    print("going to call vector_find")
    documents = collection.vector_find(reference_embeddings,limit=3)
    #print(documents)
    print("going to call show_recommendations")
    show_recommendations(documents)


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving myntrashirt1.jpeg to myntrashirt1.jpeg


In [ ]:
#reference_image = "/content/sample_image1.jpeg"

#reference_image = "/content/zaratrouser1.jpeg"
#reference_image = "/content/zara_shirt1.jpeg"
#reference_image = "/content/armani1.jpeg"
reference_image = "/content/myntrashirt1.jpeg"
#reference_image = "/content/myntrapant1.jpg"
#reference_image = "/content/zara_workout1.jpeg"


In [ ]:
print(reference_image)
find_similar_items(reference_image)

/content/myntrashirt1.jpeg
 ready to load file /content/myntrashirt1.jpeg
img_bytes type : <class 'bytes'>
img type : <class 'vertexai.vision_models.Image'>
Reference image:
Selected Categories: None
going to call get_img_embeddings
embedding: MultiModalEmbeddingResponse(_prediction_response=Prediction(predictions=[{'imageEmbedding': [-0.0236896016, 0.0356282182, 0.0696170405, 0.00501603074, -0.00900954287, 0.0232526734, 0.0150296269, 0.0357618034, 0.0171036739, -0.0246649235, -0.0250481796, 0.00743578607, 0.0135792354, 0.0747958422, -0.0159286503, -0.00224923738, 0.0253776684, 0.000489564438, 0.0398099236, -0.00281910738, 0.0858200192, 0.00271397363, -0.0458734892, 0.0122623602, -0.0548906513, 0.0120308483, -0.0101645, -0.0116291856, -0.0188627243, -0.00851001311, 0.0100838337, 0.0090680616, -0.00868586358, 0.0398379713, 0.00501281349, 0.00644221297, -0.0144699337, -0.00575962709, 0.00546239875, -0.0193713661, 0.00576382224, -0.00174730248, -0.00503114844, -0.0209484342, 0.00011003116

file1.jpg



Recommendation : 2
GEOMETRIC PRINT SWIMMING TRUNKS $41.86 , Gender: men
Swimming trunks made of technical fabric with a velvety finish. Elasticated waistband with drawstrings. Front pockets and pocket detail at the waist. Rear welt pocket. Lining. Includes a bag in the same fabric.Length of inner leg seam: 13 cm. / 5.1″Length of outer leg seam: 36 cm. / 14.1″View more
https://www.zara.com/in/en/geometric-print-swimming-trunks-p02634306.html
Similarity score:  0.6887832
https://static.zara.net/photos///2023/I/0/2/p/2634/306/401/2/w/448/2634306401_1_1_1.jpg?ts=1689157213064


file1.jpg



Recommendation : 3
WORKOUT T-SHIRT $32.06 , Gender: men
T-shirt made of lightweight textured stretch fabric that moves in every direction.- Round neck T-shirt with short sleeves.- Printed logo detail on the hem.Zara Athleticz.View more
https://www.zara.com/in/en/workout-t-shirt-p05320450.html
Similarity score:  0.6876066
https://static.zara.net/photos///2023/I/0/2/p/5320/450/250/2/w/448/5320450250_1_1_1.jpg?ts=1688985619667


file1.jpg


In [ ]:
#to find python version
!python --version

Python 3.10.12


In [ ]:
#to list all python packages installed with version
#!pip list